In [1]:
from sklearn.model_selection import train_test_split
import os
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
import sys
import math
from nltk.stem import PorterStemmer
from sklearn.datasets import load_iris
from sklearn.naive_bayes import MultinomialNB
from scipy.sparse import lil_matrix
from sklearn.feature_extraction.text import TfidfTransformer
import nltk
from nltk.corpus import stopwords
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
ps = PorterStemmer()
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings 
  
warnings.filterwarnings(action = 'ignore') 
  
import gensim 
from gensim.models import Word2Vec 
from keras.models import Sequential
from keras import layers



Using TensorFlow backend.


In [2]:
stop_words = set(stopwords.words('english'))

In [3]:
df=pd.read_csv("Dataset.csv")
df=df.drop(['Unnamed: 2'], axis = 1)
coaData = pd.read_csv("coa.csv")
nlpData = pd.read_csv("1.csv")

In [4]:
extractedsentences= df['Sentence']
labelSentence = df['Validity']
coaSentences = coaData["Sentence"]
nlpSentences = nlpData["Sentence"]

## TF-IDF

In [5]:
vectorizer = TfidfVectorizer(stop_words="english")
X_train_tfidf = vectorizer.fit_transform(extractedsentences)
coa_test_tfidf = vectorizer.transform(coaSentences)
nlp_test_tfidf = vectorizer.transform(nlpSentences)

y = np.array(labelSentence)
y= y.astype('int')


# Decision Tree

In [6]:
accuracy = []
k = np.arange(2,8)
for depth in k:
    # train data
    clf = DecisionTreeClassifier(max_depth=depth, random_state=1)
    clf.fit(X_train_tfidf,y)

    # test data
#     print("NLP Sentences")
    prediction= clf.predict(nlp_test_tfidf)
#     for j in range (len(prediction)):
#         if(prediction[j]==1):
#             print(nlpSentences[j])
#     print()
#     print("COA Sentences")
    prediction= clf.predict(coa_test_tfidf)
#     for j in range (len(prediction)):
#         if(prediction[j]==1):
#             print(coaSentences[j])
#     print()

# Polynomial SVM (Expermient with C)

In [7]:
accuracy = []
k = np.arange(2,8)
for c in k:
    classifier= svm.SVC(kernel= 'poly',degree=2, C= c, decision_function_shape='ovr')
    classifier.fit(X_train_tfidf,y)
    
#     print("NLP Sentences")
    prediction= classifier.predict(nlp_test_tfidf)
#     for j in range (len(prediction)):
#         if(prediction[j]==1):
#             print(nlpSentences[j])
#     print()
#     print("COA Sentences")
    prediction= classifier.predict(coa_test_tfidf)
#     for j in range (len(prediction)):
#         if(prediction[j]==1):
#             print(coaSentences[j])
#     print()

# Polynomial SVM (Experiment with Degree)

In [8]:
accuracy = []
k = np.arange(2,4)
for degree in k:
    classifier= svm.SVC(kernel= 'poly',degree=degree, C= 2, decision_function_shape='ovr')
    classifier.fit(X_train_tfidf,y)
#     print("NLP Sentences")
    prediction= classifier.predict(nlp_test_tfidf)
#     for j in range (len(prediction)):
#         if(prediction[j]==1):
#             print(nlpSentences[j])
#     print()
#     print("COA Sentences")
    prediction= classifier.predict(coa_test_tfidf)
#     for j in range (len(prediction)):
#         if(prediction[j]==1):
#             print(coaSentences[j])
#     print()

# Linear SVM

In [9]:
accuracy = []
k = np.arange(2,10)
for c in k:
    classifier= svm.SVC(kernel= 'linear', C= c, decision_function_shape='ovr')
    classifier.fit(X_train_tfidf,y)
#     print("NLP Sentences")
    prediction= classifier.predict(nlp_test_tfidf)
#     for j in range (len(prediction)):
#         if(prediction[j]==1):
#             print(nlpSentences[j])
#     print()
#     print("COA Sentences")
    prediction= classifier.predict(coa_test_tfidf)
#     for j in range (len(prediction)):
#         if(prediction[j]==1):
#             print(coaSentences[j])
#     print()

# KNeighbours

In [10]:
accuracy = []
k = np.arange(3,10)
for numberOfNeighbours in k:
    knn = KNeighborsClassifier(n_neighbors=numberOfNeighbours, metric='euclidean')
    knn.fit(X_train_tfidf, y)
#     print("NLP Sentences")
    prediction= knn.predict(nlp_test_tfidf)
#     for j in range (len(prediction)):
#         if(prediction[j]==1):
#             print(nlpSentences[j])
#     print()
#     print("COA Sentences")
    prediction= knn.predict(coa_test_tfidf)
#     for j in range (len(prediction)):
#         if(prediction[j]==1):
#             print(coaSentences[j])
#     print()

# Random Forest

In [11]:
accuracy = []
k = np.arange(2,10)
for depth in k:
    clf = RandomForestClassifier(max_depth=depth, random_state=1)
    clf.fit(X_train_tfidf, y)
#     print("NLP Sentences")
    prediction= clf.predict(nlp_test_tfidf)
#     for j in range (len(prediction)):
#         if(prediction[j]==1):
#             print(nlpSentences[j])
#     print()
#     print("COA Sentences")
    prediction= clf.predict(coa_test_tfidf)
#     for j in range (len(prediction)):
#         if(prediction[j]==1):
#             print(coaSentences[j])
#     print()

## Word-2-Vec

In [12]:


from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

ps= PorterStemmer()
tokenized_text= [sentence.split() for sentence in extractedsentences]
data= []
for tokenList in tokenized_text:
    temp =[]
    for token in tokenList:
        if token not in stop_words:
            temp.append(ps.stem(token))
    data.append(temp)

path = get_tmpfile("word2vec.model")
model = Word2Vec(data, size=100, window=5, min_count=2, workers=4)
model.save("word2vec.model")

model = Word2Vec.load("word2vec.model")

tokenized_text= [sentence.split() for sentence in extractedsentences]
data= []
for tokenList in tokenized_text:
    zerosList= [int(0) for i in range(100)]
    temp =[]
    temp.append(zerosList)
    for token in tokenList:
        if token not in stop_words and token in model:
            temp.append(model[ps.stem(token)])
    temp= np.array(temp)
    temp= np.mean(temp, axis=0)
    data.append(temp)

xtrain = np.zeros((len(data),100))
i=-1
j=-1
for values in data:
    i+=1
    j=-1
    for q in range(values.shape[0]):
        xtrain[i][j]=values[q]
X_train=xtrain 

tokenized_text= [sentence.split() for sentence in coaSentences]
data= []
for tokenList in tokenized_text:
    zerosList= [int(0) for i in range(100)]
    temp =[]
    temp.append(zerosList)
    for token in tokenList:
        if token not in stop_words and token in model:
            temp.append(model[ps.stem(token)])
    temp= np.array(temp)
    temp= np.mean(temp, axis=0)
    data.append(temp)

xtrain = np.zeros((len(data),100))
i=-1
j=-1
for values in data:
    i+=1
    j=-1
    for q in range(values.shape[0]):
        xtrain[i][j]=values[q]
coa_test=xtrain 

tokenized_text= [sentence.split() for sentence in nlpSentences]
data= []
for tokenList in tokenized_text:
    zerosList= [int(0) for i in range(100)]
    temp =[]
    temp.append(zerosList)
    for token in tokenList:
        if token not in stop_words and token in model:
            temp.append(model[ps.stem(token)])
    temp= np.array(temp)
    temp= np.mean(temp, axis=0)
    data.append(temp)

xtrain = np.zeros((len(data),100))
i=-1
j=-1
for values in data:
    i+=1
    j=-1
    for q in range(values.shape[0]):
        xtrain[i][j]=values[q]
nlp_test=xtrain 

y = np.array(labelSentence)
y= y.astype('int')

# Decision Tree

In [13]:
accuracy = []
k = np.arange(2,8)
for depth in k:
    # train data
    clf = DecisionTreeClassifier(max_depth=depth, random_state=1)
    clf.fit(X_train,y)

    # test data
#     print("NLP Sentences")
    prediction= clf.predict(nlp_test)
#     for j in range (len(prediction)):
#         if(prediction[j]==1):
#             print(nlpSentences[j])
#     print()
#     print("COA Sentences")
    prediction= clf.predict(coa_test)
#     for j in range (len(prediction)):
#         if(prediction[j]==1):
#             print(coaSentences[j])
#     print()

# Polynomial SVM (Expermient with C)

In [14]:
accuracy = []
k = np.arange(2,8)
for c in k:
    classifier= svm.SVC(kernel= 'poly',degree=2, C= c, decision_function_shape='ovr')
    classifier.fit(X_train,y)
    
#     print("NLP Sentences")
    prediction= classifier.predict(nlp_test)
#     for j in range (len(prediction)):
#         if(prediction[j]==1):
#             print(nlpSentences[j])
#     print()
#     print("COA Sentences")
#     prediction= classifier.predict(coa_test)
#     for j in range (len(prediction)):
#         if(prediction[j]==1):
#             print(coaSentences[j])
#     print()

# Polynomial SVM (Experiment with Degree)

In [15]:
accuracy = []
k = np.arange(2,4)
for degree in k:
    classifier= svm.SVC(kernel= 'poly',degree=degree, C= 2, decision_function_shape='ovr')
    classifier.fit(X_train,y)
#     print("NLP Sentences")
    prediction= classifier.predict(nlp_test)
#     for j in range (len(prediction)):
#         if(prediction[j]==1):
#             print(nlpSentences[j])
#     print()
#     print("COA Sentences")
#     prediction= classifier.predict(coa_test)
#     for j in range (len(prediction)):
#         if(prediction[j]==1):
#             print(coaSentences[j])
#     print()

# Linear SVM

In [16]:
accuracy = []
k = np.arange(2,10)
for c in k:
    classifier= svm.SVC(kernel= 'linear', C= c, decision_function_shape='ovr')
    classifier.fit(X_train,y)
#     print("NLP Sentences")
    prediction= classifier.predict(nlp_test)
#     for j in range (len(prediction)):
#         if(prediction[j]==1):
#             print(nlpSentences[j])
#     print()
#     print("COA Sentences")
#     prediction= classifier.predict(coa_test)
#     for j in range (len(prediction)):
#         if(prediction[j]==1):
#             print(coaSentences[j])
#     print()

# KNeighbours

In [17]:
accuracy = []
k = np.arange(3,10)
for numberOfNeighbours in k:
    knn = KNeighborsClassifier(n_neighbors=numberOfNeighbours, metric='euclidean')
    knn.fit(X_train, y)
#     print("NLP Sentences")
#     prediction= knn.predict(nlp_test)
#     for j in range (len(prediction)):
#         if(prediction[j]==1):
#             print(nlpSentences[j])
#     print()
#     print("COA Sentences")
#     prediction= knn.predict(coa_test)
#     for j in range (len(prediction)):
#         if(prediction[j]==1):
#             print(coaSentences[j])
#     print()

# Random Forest

In [18]:
accuracy = []
k = np.arange(2,10)
for depth in k:
    clf = RandomForestClassifier(max_depth=depth, random_state=1)
    clf.fit(X_train, y)
#     print("NLP Sentences")
#     prediction= clf.predict(nlp_test)
#     for j in range (len(prediction)):
#         if(prediction[j]==1):
#             print(nlpSentences[j])
#     print()
#     print("COA Sentences")
#     prediction= clf.predict(coa_test)
#     for j in range (len(prediction)):
#         if(prediction[j]==1):
#             print(coaSentences[j])
#     print()